In [1]:
# from gan_layers import gan_network
import os
import numpy as np
from PIL import Image
import random
import cv2

In [2]:
input_dir = 'train/240p/'
tgt_dir = 'train/720p/'
img_list = os.listdir(input_dir)
assert img_list == os.listdir(tgt_dir)

input_path = input_dir + img_list[0]
input_shape_temp = np.array(Image.open(input_path)).shape

tgt_path = tgt_dir + img_list[0]
tgt_shape_temp = np.array(Image.open(tgt_path)).shape

In [3]:
from layer import UNET

Using TensorFlow backend.


In [44]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Dense
from keras.layers import Flatten, Reshape, Activation, Concatenate, Dropout, BatchNormalization


class UNET():
    def __init__(self, input_shape=(240, 426, 3), tgt_shape=(720, 1280, 3)):
        self.input_shape = input_shape
        self.tgt_shape = tgt_shape

    def down(self, input_layer, filters, pool=True):
        conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(input_layer)
        residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
        if pool:
            max_pool = MaxPooling2D()(residual)
            return max_pool, residual
        else:
            return residual

    def up_res(self, input_layer, residual, filters):
        filters= int(filters)
        upsample = UpSampling2D()(input_layer)
        upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
        concat = Concatenate(axis=3)([residual, upconv])
        conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)    
        conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
        return conv2
    
    def up_only(self, input_layer, filters):
        filters= int(filters)
        upsample = UpSampling2D()(input_layer)
        upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
        conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(upconv)    
        conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
        return conv2

    def unet(self, num_filter=64):
        input_layer = Input(shape = self.input_shape)
        layers = [input_layer]
        residuals = []

        # Down 1, 64
        down1, res1 = self.down(input_layer, num_filter)
        residuals.append(res1)
        num_filter *= 2

#         # Down 2, 128
#         down2, res2 = self.down(down1, num_filter)
#         residuals.append(res1)
#         num_filter *= 2

        # Down 3, 8
        down3 = self.down(down1, num_filter, pool=False)

        # Up 1, 128
        num_filter /= 2
        up1 = self.up_res(down3, residual=residuals[-1], filters=num_filter)

        # Up 2, 64
        num_filter /= 2
#         up2 = self.up_only(up1, filters=num_filter)
        up3 = UpSampling2D(size=(3,3))(up1)
        out = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(up3)
        model = Model(input_layer, out)

        return model


In [45]:
model = UNET().unet()

In [46]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 240, 426, 3)  0                                            
__________________________________________________________________________________________________
conv2d_118 (Conv2D)             (None, 240, 426, 64) 1792        input_17[0][0]                   
__________________________________________________________________________________________________
conv2d_119 (Conv2D)             (None, 240, 426, 64) 36928       conv2d_118[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_23 (MaxPooling2D) (None, 120, 213, 64) 0           conv2d_119[0][0]                 
____________________________________________________________________________________________

In [64]:
class ImageDataGenerator(object):
    def __init__(self, input_dir, tgt_dir):
        self.input_dir = input_dir
        self.tgt_dir = tgt_dir
        self.img_list = os.listdir(input_dir)
        assert self.img_list == os.listdir(tgt_dir)
        self.reset()

    def reset(self):
        self.inputs = []
        self.targets = []

    def flow_from_directory(self, batch_size=16):
        train_imges = random.sample(self.img_list, batch_size)
        while True:
            for img in train_imges:
                input_path = input_dir + img
                tgt_path = tgt_dir + img

                X = (np.array(Image.open(input_path)) - 127.5) / 127.5
                Y = (np.array(Image.open(tgt_path)) - 127.5) / 127.5

                X_list.append(X.reshape([1, X.shape[0], X.shape[1], X.shape[2]]))
                Y_list.append(Y.reshape([1, Y.shape[0], Y.shape[1], Y.shape[2]]))
    
            inputs = np.vstack(X_list)
            targets = np.vstack(Y_list)

            yield inputs, targets

In [ ]:
# 